## Monitoring Machine Learning Models
Before deploying a machine learning model to production, it's important to instrument monitoring metrics in order to ensure the model will perform as it did during training.

According to a paper from Google Research titled [What’s your ML Test Score? A rubric for ML production systems](https://storage.googleapis.com/pub-tools-public-publication-data/pdf/45742.pdf), here are some reasons to monitor machine learning models in production:

+ Monitor distributions of each feature used in the model for prediction to match your expectations. 
  + If feature A takes on values 1 to 5 during training, ensure that distribution holds true for new unseen data.   +   
  + Similarily, if feature B contains movies from last summer 'Transformers', 'Baywatch' & 'Guardians of the Galaxy' during training, how does your model handle a new movie from this summer like 'Incredibles 2'? 
  
  + Feature distributions often change due to upstream or external dependencies like third party data changes.

+ What happens to your model if a required feature is missing? Real world systems can change, partially or completely fail over time, how will you know without instrumenting monitoring?  

+ A/B testing is often used for a variety of reasons, monitoring allows us to keep track of the experiment results. 

+ Compute computational time of feature engineering & model prediction

+ Compute model prediction health metrics including transactions per second (successes & failures)

We will be extending our Dynamic Inference example using **[clipper.ai](http://clipper.ai/)** to demonstrate how to instrument monitoring in your machine learning models.

Overview  
+ Instrument metrics in production `predict(...)` method 

References:

+ [clipper.ai documentation](http://clipper.ai/)

+ [clipper @github](https://github.com/ucbrise/clipper)

### Model Training
Before we can serve a model, we must first train it. Model training is an iterative process often persisting artifacts to disk. After model has been trained, we can load the model back into memory (RAM) when serving.

In [1]:
from __future__ import print_function
import sys
sys.path.insert(0, '../../clipper_admin')
from clipper_admin import ClipperConnection, DockerContainerManager
from clipper_admin.deployers import python as python_deployer
import json
import requests
from datetime import datetime
import time
import numpy as np
import signal
import sys


def predict(addr, x, batch=False):
    url = "http://%s/simple-example/predict" % addr

    if batch:
        req_json = json.dumps({'input_batch': x})
    else:
        req_json = json.dumps({'input': list(x)})

    headers = {'Content-type': 'application/json'}
    start = datetime.now()
    r = requests.post(url, headers=headers, data=req_json)
    end = datetime.now()
    latency = (end - start).total_seconds() * 1000.0
    print("'%s', %f ms" % (r.text, latency))


def feature_sum(xs):
    return [str(sum(x)) for x in xs]


# Stop Clipper on Ctrl-C
def signal_handler(signal, frame):
    print("Stopping Clipper...")
    clipper_conn = ClipperConnection(DockerContainerManager())
    clipper_conn.stop_all()
    sys.exit(0)


def produce_query_arr_for_ms(ms):
    size = int(ms * 8000)  ## Use python sum, scale linearly.
    return np.random.random(size)


def fizz_buzz(i):
    if i % 15 == 0:
        return produce_query_arr_for_ms(200)
    elif i % 5 == 0:
        return produce_query_arr_for_ms(100)
    elif i % 3 == 0:
        return produce_query_arr_for_ms(50)
    else:
        return produce_query_arr_for_ms(10)

In [3]:
signal.signal(signal.SIGINT, signal_handler)
clipper_conn = ClipperConnection(DockerContainerManager())
clipper_conn.start_clipper()
print("Starting Clipper")
python_deployer.create_endpoint(
    clipper_conn, "simple-example", "doubles", feature_sum, num_replicas=2)
time.sleep(2)
print("Starting Prediction")

try:
    counter = 0
    while True:
        print(counter)
        predict(clipper_conn.get_query_addr(), fizz_buzz(counter))
        counter += 1
        time.sleep(0.2)
except Exception as e:
    clipper_conn.stop_all()

18-08-23:19:13:20 INFO     [docker_container_manager.py:151] [default-cluster] Starting managed Redis instance in Docker
18-08-23:19:13:23 INFO     [docker_container_manager.py:229] [default-cluster] Metric Configuration Saved at /private/var/folders/kv/w56d6z9j4c79zvw8c8jsn6hw0000gn/T/tmpwdmy3e49.yml
18-08-23:19:13:24 INFO     [clipper_admin.py:138] [default-cluster] Clipper is running
18-08-23:19:13:24 INFO     [clipper_admin.py:215] [default-cluster] Application simple-example was successfully registered
18-08-23:19:13:24 INFO     [deployer_utils.py:41] Saving function to /var/folders/kv/w56d6z9j4c79zvw8c8jsn6hw0000gn/T/tmp2iudv061clipper
18-08-23:19:13:24 INFO     [deployer_utils.py:51] Serialized and supplied predict function
18-08-23:19:13:24 INFO     [python.py:192] Python closure saved
18-08-23:19:13:24 INFO     [python.py:206] Using Python 3.6 base image
18-08-23:19:13:24 INFO     [clipper_admin.py:467] [default-cluster] Building model Docker image with model data from /var/fo

Starting Clipper


18-08-23:19:13:25 INFO     [clipper_admin.py:472] [default-cluster] Step 1/2 : FROM clipper/python36-closure-container:develop
18-08-23:19:13:25 INFO     [clipper_admin.py:472] [default-cluster]  ---> 0fac6e6e8242
18-08-23:19:13:25 INFO     [clipper_admin.py:472] [default-cluster] Step 2/2 : COPY /var/folders/kv/w56d6z9j4c79zvw8c8jsn6hw0000gn/T/tmp2iudv061clipper /model/
18-08-23:19:13:25 INFO     [clipper_admin.py:472] [default-cluster]  ---> 5beb7b9415ec
18-08-23:19:13:25 INFO     [clipper_admin.py:472] [default-cluster] Successfully built 5beb7b9415ec
18-08-23:19:13:25 INFO     [clipper_admin.py:472] [default-cluster] Successfully tagged default-cluster-simple-example:1
18-08-23:19:13:25 INFO     [clipper_admin.py:474] [default-cluster] Pushing model Docker image to default-cluster-simple-example:1
18-08-23:19:13:26 INFO     [docker_container_manager.py:353] [default-cluster] Found 0 replicas for simple-example:1. Adding 2
18-08-23:19:13:34 INFO     [clipper_admin.py:651] [default-c

Starting Prediction
0
'{"query_id":0,"output":800617.5344384846,"default":false}', 1075.436000 ms
1
'{"query_id":1,"output":40073.47667134216,"default":false}', 81.898000 ms
2
'{"query_id":2,"output":40179.55371382129,"default":false}', 56.110000 ms
3
'{"query_id":3,"output":200018.33858941205,"default":false}', 221.418000 ms
4
'{"query_id":4,"output":39868.900389668896,"default":false}', 53.996000 ms
5
'{"query_id":5,"output":400160.18263414429,"default":false}', 595.396000 ms
6
'{"query_id":6,"output":200070.06922581289,"default":false}', 541.763000 ms
7
'{"query_id":7,"output":39900.666594653616,"default":false}', 119.882000 ms
8
'{"query_id":8,"output":40038.02149527564,"default":false}', 98.743000 ms
9
'{"query_id":9,"output":200078.29155371697,"default":false}', 1142.243000 ms
10
'{"query_id":10,"output":400134.414537437,"default":false}', 510.269000 ms
11
'{"query_id":11,"output":39956.02302033329,"default":false}', 92.557000 ms
12
'{"query_id":12,"output":200332.97951939456,"de

'{"query_id":103,"output":39816.3399747693,"default":false}', 164.671000 ms
104
'{"query_id":104,"output":40048.92336684931,"default":false}', 60.198000 ms
105
'{"query_id":105,"output":"None","default":true,"default_explanation":"Failed to retrieve a prediction response within the specified latency SLO"}', 3546.974000 ms
106
'{"query_id":106,"output":39982.55388222861,"default":false}', 194.091000 ms
107
'{"query_id":107,"output":39957.57870371034,"default":false}', 105.953000 ms
108
'{"query_id":108,"output":200254.2307967595,"default":false}', 325.994000 ms
109
'{"query_id":109,"output":39826.63906216726,"default":false}', 71.408000 ms
110
'{"query_id":110,"output":399805.881432954,"default":false}', 398.743000 ms
111
'{"query_id":111,"output":200153.1100054572,"default":false}', 211.432000 ms
112
'{"query_id":112,"output":39924.76830551944,"default":false}', 51.782000 ms
113
'{"query_id":113,"output":39987.431225298744,"default":false}', 76.664000 ms
114
'{"query_id":114,"output":1

...

'{"query_id":6591,"output":199930.65537130476,"default":false}', 204.553000 ms
6592
'{"query_id":6592,"output":39983.80211794424,"default":false}', 36.264000 ms
6593
'{"query_id":6593,"output":40124.18623892056,"default":false}', 50.786000 ms
6594
'{"query_id":6594,"output":199794.25727984354,"default":false}', 168.312000 ms
6595
'{"query_id":6595,"output":400079.30534536347,"default":false}', 410.360000 ms
6596
'{"query_id":6596,"output":40018.10341156231,"default":false}', 64.847000 ms
6597
'{"query_id":6597,"output":199825.46291108556,"default":false}', 254.220000 ms
6598
'{"query_id":6598,"output":40002.17370984342,"default":false}', 36.726000 ms
6599
'{"query_id":6599,"output":40119.60495218668,"default":false}', 56.699000 ms
6600
'{"query_id":6600,"output":800666.9825513191,"default":false}', 1044.232000 ms
6601
'{"query_id":6601,"output":39973.81355473796,"default":false}', 42.431000 ms
6602
'{"query_id":6602,"output":39962.10148359784,"default":false}', 51.065000 ms
6603
'{"que

6691
'{"query_id":6691,"output":39980.60217793594,"default":false}', 36.867000 ms
6692
'{"query_id":6692,"output":40090.612856528736,"default":false}', 40.626000 ms
6693
'{"query_id":6693,"output":199831.8192784437,"default":false}', 227.570000 ms
6694
'{"query_id":6694,"output":39898.14495196828,"default":false}', 43.334000 ms
6695
'{"query_id":6695,"output":399922.7307802875,"default":false}', 440.552000 ms
6696
'{"query_id":6696,"output":199932.56719454946,"default":false}', 152.046000 ms
6697
'{"query_id":6697,"output":40066.82195965442,"default":false}', 32.049000 ms
6698
'{"query_id":6698,"output":40047.01644563091,"default":false}', 37.170000 ms
6699
'{"query_id":6699,"output":200186.8446795196,"default":false}', 177.918000 ms
6700
'{"query_id":6700,"output":400005.9002607285,"default":false}', 452.950000 ms
6701
'{"query_id":6701,"output":40092.64265750322,"default":false}', 57.525000 ms
6702
'{"query_id":6702,"output":199807.14286573104,"default":false}', 235.681000 ms
6703
'{

18-08-23:22:57:17 INFO     [clipper_admin.py:1278] [default-cluster] Stopped all Clipper cluster and all model containers


SystemExit: 0

/Users/arm/anaconda3/envs/py36_oreilly_ml_prod_course/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2971: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## To Shutdown Clipper
Clipper creates docker containers for various microservices. The following commands will shutdown clipper gracefully.

In [ ]:
print("Shutting down Clipper connection.")
clipper_conn.stop_all()

In [ ]:
# stop all containers:
!docker rm $(docker ps -a -q)

## Launch Dashboard

**clipper** ships with [Prometheus](https://prometheus.io/) as the metric tracking system for monitoring ML models in production. When you start a Clipper cluster, a Prometheus instance is automatically started for you. **clipper** provides latency and throughput metrics by default. In particular, we provide latency and throughput metrics across several processing stages.



![alt text](../assets/images/launch_graphana.png)

You can visit `http://localhost:3000` to see the web user interface. Default username & password is `admin`.

![alt text](../assets/images/graphana_login.png)

Pick a new password which is atleast 7 characters.

![alt text](../assets/images/graphana_change_password.png)

After you change your password, you'll see an empty dashboard...

![alt text](../assets/images/graphana_logged_in.png)

After Login, Click `Home` -> `Import Dashboard` -> `Upload json File` -> `Clipper_Dashboard.json`

`Clipper_Dashboard.json` is located [here](../monitoring/Clipper_Dashboard.json).

![alt text](../assets/images/graphana_import_json.png)

Click `Import Dashboard` 

![alt text](../assets/images/graphana_import_dashboard.png)

Click `Upload json File` then select `Clipper_Dashboard.json`

![alt text](../assets/images/graphana_dashboard.png)